## Install and import Libraries

In [2]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium

print('Packages installed.')

     |████████████████████████████████| 102kB 6.5MB/s ta 0:00:011
     |████████████████████████████████| 92kB 6.1MB/s eta 0:00:011
Packages installed.


In [3]:
import numpy as np #to handle data in a vectorized manner
import pandas as pd #for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 



In [4]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
print('Libraries has been imported.')

Libraries has been imported.


## Scapping Page

In [6]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


## Collecting data

In [7]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [8]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [9]:
# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)

In [10]:
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A,Not assigned,M2A,43.648690,-79.385440
1,M3A,North York,Parkwoods,43.752935,-79.335641
2,M4A,North York,Victoria Village,43.728102,-79.311890
3,M5A,Downtown Toronto,Regent Park / Harbourfront,43.650964,-79.353041
4,M6A,North York,Lawrence Manor / Lawrence Heights,43.723265,-79.451211
5,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.661790,-79.389390
6,M8A,Not assigned,M9A,43.648690,-79.385440
7,M1B,Scarborough,Malvern / Rouge,43.808626,-79.189913
8,M2B,Not assigned,M3B,43.648690,-79.385440
9,M4B,East York,Parkview Hill / Woodbine Gardens,43.707193,-79.311529


In [11]:
neighbors.shape

(135, 5)

In [26]:
VERSION = '20200501' # Foursquare API version

neighborhood_name = neighbors.loc[0, 'Neighborhood'] # neighborhood name
neighborhood_latitude = neighbors.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighbors.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

#Define Radius
radius = 500 
#Limit of the number of venues returned by Foursquare API
LIMIT = 100 

# url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID, 
#     CLIENT_SECRET, 
#     VERSION, 
#     43.67635739999999, 
#     79.2930312, 
#     radius, 
#     LIMIT)

url = 'https://api.foursquare.com/v2/venues/explore?&client_id=E0IMNYNCMI4Z5UR4SSDPKX44VAOMPIF1DN52LK5QQ110Y3BZ&client_secret=4WXFQTMEH1DTBZAZXG2ZIQF4BEFUWPJA5WVBIBOHFUI5SAG4&v=20200501&ll=43.728102,-79.3118901&radius=500&limit=100'

results = requests.get(url).json()
results

Latitude and longitude values of M2A
 are 43.648690000000045, -79.38543999999996.


{'meta': {'code': 200, 'requestId': '5eb0e61a69babe001bd7f65b'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 43.7326020045, 'lng': -79.30567445207},
   'sw': {'lat': 43.7236019955, 'lng': -79.31810574793}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bbe904a85fbb713420d7167',
       'name': 'Tim Hortons',
       'location': {'address': '1733 Eglinton Ave East',
        'crossStreet': 'at Bermondsey Rd',
        'lat': 43.725516631714754,
        'lng': -79.31310251366091,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.725516631714754,
          'lng': -79.31310251366091}],
        'distance': 303,
        'postalCode'

### From the Foursquare lab in the previous module........we know that all the information is in the items key
### Before we proceed...... let borrow the get_category_type function from the Foursquare lab
## function that extract the category of the venue

In [27]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Hearn Generating Station,Government Building,43.646847,-79.335144
1,Waterfront Night Market,Night Market,43.646299,-79.333682
2,Urban Warrior Toronto,Athletics & Sports,43.645486,-79.334390


In [28]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


### create the API request URL
###         url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}
###          &client_secret={}&v=   {}&ll={},{}&radius={}&limit={}'.format(
###            CLIENT_ID, 
###            CLIENT_SECRET, 
###            VERSION, 
###            lat, 
###             lng, 
###             radius, 
###            LIMIT)
        

In [29]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
toronto_venues = getNearbyVenues(names=neighborhoods,
                                 latitudes=neighbors['Latitude'],
                                 longitudes=neighbors['Longitude'])

M2A

Parkwoods

Victoria Village

Regent Park / Harbourfront

Lawrence Manor / Lawrence Heights

Queen's Park / Ontario Provincial Government

M9A

Malvern / Rouge

M3B

Parkview Hill / Woodbine Gardens

Garden District / Ryerson

Glencairn

M8B

West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale

Rouge Hill / Port Union / Highland Creek

M3C

Woodbine Heights

St. James Town

Humewood-Cedarvale

M8C

Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood

Guildwood / Morningside / West Hill

M3E

The Beaches

Berczy Park

Caledonia-Fairbanks

M8E

M1G

M3G

Leaside

Central Bay Street

Christie

M8G

M1H

Hillcrest Village

Bathurst Manor / Wilson Heights / Downsview North

Thorncliffe Park

Richmond / Adelaide / King

Dufferin / Dovercourt Village

M8H

M1J

Fairview / Henry Farm / Oriole

Northwood Park / York University

East Toronto

Harbourfront East / Union Station / Toronto Islands

Little Portugal / Trinity

M8J

M1K

Bayview Village

Downsvie

In [31]:
print(toronto_venues.shape)
toronto_venues.head()

(810, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M2A,43.64869,-79.38544,Tim Hortons,43.725517,-79.313103,Coffee Shop
1,M2A,43.64869,-79.38544,Portugril,43.725819,-79.312785,Portuguese Restaurant
2,M2A,43.64869,-79.38544,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
3,M2A,43.64869,-79.38544,Pizza Nova,43.725824,-79.312860,Pizza Place
4,M2A,43.64869,-79.38544,Wigmore Park,43.731023,-79.310771,Park


In [32]:
# check how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,6,6,6,6,6,6
Alderwood / Long Branch,6,6,6,6,6,6
Bathurst Manor / Wilson Heights / Downsview North,6,6,6,6,6,6
Bayview Village,6,6,6,6,6,6
Bedford Park / Lawrence Manor East,6,6,6,6,6,6
Berczy Park,6,6,6,6,6,6
Birch Cliff / Cliffside West,6,6,6,6,6,6
Brockton / Parkdale Village / Exhibition Place,6,6,6,6,6,6
Business reply mail Processing Centre,6,6,6,6,6,6


In [33]:
# find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 6 uniques categories.


In [34]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Coffee Shop,French Restaurant,Intersection,Park,Pizza Place,Portuguese Restaurant
0,M2A,1,0,0,0,0,0
1,M2A,0,0,0,0,0,1
2,M2A,0,0,1,0,0,0
3,M2A,0,0,0,0,1,0
4,M2A,0,0,0,1,0,0


In [35]:
# examine the new dataframe size.

toronto_onehot.shape

(810, 7)

In [36]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Coffee Shop,French Restaurant,Intersection,Park,Pizza Place,Portuguese Restaurant
0,Agincourt,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667
1,Alderwood / Long Branch,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667
2,Bathurst Manor / Wilson Heights / Downsview No...,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667
3,Bayview Village,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667
4,Bedford Park / Lawrence Manor East,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667
5,Berczy Park,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667
6,Birch Cliff / Cliffside West,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667
7,Brockton / Parkdale Village / Exhibition Place,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667
8,Business reply mail Processing Centre,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667
9,CN Tower / King and Spadina / Railway Lands / ...,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667


In [37]:
# confirm the new size

toronto_grouped.shape

(131, 7)

In [38]:
# print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt
----
               venue  freq
0        Coffee Shop  0.17
1  French Restaurant  0.17
2       Intersection  0.17
3               Park  0.17
4        Pizza Place  0.17


----Alderwood / Long Branch
----
               venue  freq
0        Coffee Shop  0.17
1  French Restaurant  0.17
2       Intersection  0.17
3               Park  0.17
4        Pizza Place  0.17


----Bathurst Manor / Wilson Heights / Downsview North
----
               venue  freq
0        Coffee Shop  0.17
1  French Restaurant  0.17
2       Intersection  0.17
3               Park  0.17
4        Pizza Place  0.17


----Bayview Village
----
               venue  freq
0        Coffee Shop  0.17
1  French Restaurant  0.17
2       Intersection  0.17
3               Park  0.17
4        Pizza Place  0.17


----Bedford Park / Lawrence Manor East
----
               venue  freq
0        Coffee Shop  0.17
1  French Restaurant  0.17
2       Intersection  0.17
3               Park  0.17
4        Pizza Place  0.17




4        Pizza Place  0.17


----Little Portugal / Trinity
----
               venue  freq
0        Coffee Shop  0.17
1  French Restaurant  0.17
2       Intersection  0.17
3               Park  0.17
4        Pizza Place  0.17


----M1G
----
               venue  freq
0        Coffee Shop  0.17
1  French Restaurant  0.17
2       Intersection  0.17
3               Park  0.17
4        Pizza Place  0.17


----M1H
----
               venue  freq
0        Coffee Shop  0.17
1  French Restaurant  0.17
2       Intersection  0.17
3               Park  0.17
4        Pizza Place  0.17


----M1J
----
               venue  freq
0        Coffee Shop  0.17
1  French Restaurant  0.17
2       Intersection  0.17
3               Park  0.17
4        Pizza Place  0.17


----M1K
----
               venue  freq
0        Coffee Shop  0.17
1  French Restaurant  0.17
2       Intersection  0.17
3               Park  0.17
4        Pizza Place  0.17


----M1L
----
               venue  freq
0        Coffee Shop  0.

               venue  freq
0        Coffee Shop  0.17
1  French Restaurant  0.17
2       Intersection  0.17
3               Park  0.17
4        Pizza Place  0.17


----New Toronto / Mimico South / Humber Bay Shores
----
               venue  freq
0        Coffee Shop  0.17
1  French Restaurant  0.17
2       Intersection  0.17
3               Park  0.17
4        Pizza Place  0.17


----North Park / Maple Leaf Park / Upwood Park
----
               venue  freq
0        Coffee Shop  0.17
1  French Restaurant  0.17
2       Intersection  0.17
3               Park  0.17
4        Pizza Place  0.17


----Northwest
----
               venue  freq
0        Coffee Shop  0.17
1  French Restaurant  0.17
2       Intersection  0.17
3               Park  0.17
4        Pizza Place  0.17


----Northwood Park / York University
----
               venue  freq
0        Coffee Shop  0.17
1  French Restaurant  0.17
2       Intersection  0.17
3               Park  0.17
4        Pizza Place  0.17


----Old Mil

In [39]:
# sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Agincourt,Portuguese Restaurant,Pizza Place,Park
1,Alderwood / Long Branch,Portuguese Restaurant,Pizza Place,Park
2,Bathurst Manor / Wilson Heights / Downsview No...,Portuguese Restaurant,Pizza Place,Park
3,Bayview Village,Portuguese Restaurant,Pizza Place,Park
4,Bedford Park / Lawrence Manor East,Portuguese Restaurant,Pizza Place,Park


In [41]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [42]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M1A,Not assigned,M2A,43.648690,-79.385440,0,Portuguese Restaurant,Pizza Place,Park
1,M3A,North York,Parkwoods,43.752935,-79.335641,0,Portuguese Restaurant,Pizza Place,Park
2,M4A,North York,Victoria Village,43.728102,-79.311890,0,Portuguese Restaurant,Pizza Place,Park
3,M5A,Downtown Toronto,Regent Park / Harbourfront,43.650964,-79.353041,0,Portuguese Restaurant,Pizza Place,Park
4,M6A,North York,Lawrence Manor / Lawrence Heights,43.723265,-79.451211,0,Portuguese Restaurant,Pizza Place,Park


In [43]:
# Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[43.67635739999999, -79.2930312], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [44]:
# CLUSTER 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Not assigned,0,Portuguese Restaurant,Pizza Place,Park
1,North York,0,Portuguese Restaurant,Pizza Place,Park
2,North York,0,Portuguese Restaurant,Pizza Place,Park
3,Downtown Toronto,0,Portuguese Restaurant,Pizza Place,Park
4,North York,0,Portuguese Restaurant,Pizza Place,Park
5,Downtown Toronto,0,Portuguese Restaurant,Pizza Place,Park
6,Not assigned,0,Portuguese Restaurant,Pizza Place,Park
7,Scarborough,0,Portuguese Restaurant,Pizza Place,Park
8,Not assigned,0,Portuguese Restaurant,Pizza Place,Park
9,East York,0,Portuguese Restaurant,Pizza Place,Park
